In [ ]:
gameFolder = "DOA++-MAME"

In [ ]:
import sys, os
import time
timeDepSeed = int((time.time()-int(time.time()-0.5))*1000)

sys.path.append(os.path.join(os.path.abspath(''), '../'))   
sys.path.append(os.path.join(os.path.abspath(''), '../../../games',gameFolder))   

tensorBoardFolder = "./ppo2_TB_CustCnnLnLstm/"
modelFolder = "ppo2_Model_CustCnnLnLstm/"

os.makedirs(modelFolder, exist_ok=True)

In [ ]:
from makeDiambraEnv import *

import tensorflow as tf

%load_ext tensorboard

from customPolicies.customCnnLstmPolicy import *

from stable_baselines import PPO2
from stable_baselines.common.evaluation import evaluate_policy

In [ ]:
diambraKwargs = {}
diambraKwargs["roms_path"] = "../../../roms/MAMEToolkit/roms/"
diambraKwargs["binary_path"] = "../../../../customMAME/"
diambraKwargs["player"] = "P1"
diambraKwargs["frame_ratio"] = 3
#diambraKwargs["render"] =True
#diambraKwargs["throttle"] = False
#diambraKwargs["sound"] = False 
#diambraKwargs["character"] ="Random"
diambraKwargs["character"] = "Kasumi"

wrapperKwargs = {}
wrapperKwargs["frame_stack"] = 4
wrapperKwargs["clip_rewards"] = False
wrapperKwargs["normalize_rewards"] = True
wrapperKwargs["scale"] = True
wrapperKwargs["scale_mod"] = 0
wrapperKwargs["hwc_obs_resize"] = [256, 256, 1]

#keyToAdd = None
keyToAdd = []
keyToAdd.append("actionsBuf")
#keyToAdd.append("player")
keyToAdd.append("healthP1")
keyToAdd.append("healthP2")
keyToAdd.append("positionP1")
keyToAdd.append("positionP2")
#keyToAdd.append("winsP1")
#keyToAdd.append("winsP2")

numEnv=1

env = make_diambra_env(diambraMame, env_prefix="Train", num_env=numEnv, seed=timeDepSeed, 
                       continue_game = -2, diambra_kwargs = diambraKwargs, 
                       wrapper_kwargs = wrapperKwargs, key_to_add = keyToAdd)

# OR 
#env = make_diambra_env(diambraMame, num_env=2, seed=0, diambra_kwargs = diambraKwargs)
# Frame-stacking with 4 frames
#env = VecFrameStack(env, n_stack=4)

In [ ]:
print("Obs_space = ", env.observation_space)
print("Obs_space type = ", env.observation_space.dtype)
print("Obs_space high = ", env.observation_space.high)
print("Obs_space low = ", env.observation_space.low)

In [ ]:
print("Act_space = ", env.action_space)
print("Act_space type = ", env.action_space.dtype)
print("Act_space n = ", env.action_space.n)

In [ ]:
policyKwargs={}
policyKwargs["n_add_info"] = 148
policyKwargs["layers"] = [128, 256]

# Initialize the model, 1 env
#model = PPO2(CustomCnnLstmPolicy, env, nminibatches=1, verbose=1,
#             tensorboard_log=tensorBoardFolder, policy_kwargs=policyKwargs, gamma = 0.94)

#OR

# Load the trained agent, 1 env
model = PPO2.load(modelFolder+"4_5Msteps_action+_add", env=env, tensorboard_log=tensorBoardFolder, 
                  policy_kwargs=policyKwargs, gamma = 0.94)

In [ ]:
# Train the agent
time_steps = 10000000
model.learn(total_timesteps=time_steps)

In [ ]:
# Save the agent
model.save(modelFolder+"5_8Msteps_action+_add")

In [ ]:
%tensorboard --logdir ./tensorBoardFolder/

In [ ]:
observation = env.reset()
states = None

while True:

    action, states = model.predict(observation, states, deterministic=False)
    action_prob = model.action_probability(observation, states)
    print("Action probabilities = ", action_prob)
    print("Max action = ", np.argmax(action_prob))
    print("Action = ", action)
    input("Pausa")
    
    observation, reward, done, info = env.step(action)
    if done:
        observation = env.reset()
        states = None
        
env.close()

In [ ]:
observation = env.reset()
states = None

cumulativeEpRew = 0.0
cumulativeEpRewAll = []
cumulativeTotRew = 0.0

maxNumEp = 100
currNumEp = 0

while currNumEp < maxNumEp:

    action, states = model.predict(observation, states, deterministic=False)
    action_prob = model.action_probability(observation, states)
    print("Action probabilities = ", action_prob)
    print("Max action = ", np.argmax(action_prob))
    print("Action = ", action)
    
    observation, reward, done, info = env.step(action)
    
    cumulativeEpRew += reward
    
    if np.any(done):
        currNumEp += 1
        print("Ep. # = ", currNumEp)
        print("Ep. Cumulative Rew # = ", cumulativeEpRew)
        sys.stdout.flush()
        cumulativeEpRewAll.append(cumulativeEpRew)
        cumulativeTotRew += cumulativeEpRew
        cumulativeEpRew = 0.0

        observation = env.reset()
        states = None

print("Mean cumulative reward = ", cumulativeTotRew/maxNumEp)    
print("Mean cumulative reward = ", np.mean(cumulativeEpRewAll))    
print("Std cumulative reward = ", np.std(cumulativeEpRewAll))   
    
env.close()